In [1]:
import contextlib
import io

f = io.StringIO()
with contextlib.redirect_stdout(f):
    import numpy as np
    import pandas as pd
    import import_ipynb
    import baseline_model
    import als
    import spectral_regularization_model
    import nuclear_norm_model
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
def recommend_anime(R, 
                    u_id,
                    df, 
                    x=5):
    
    original_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    R_df = pd.DataFrame(R, index=original_matrix.index, columns=original_matrix.columns)
    
    # find the user row of original_matrix
    user_row = original_matrix.loc[u_id]
    
    # get the anime IDs that the user has already watched (ratings > 0)
    user_watched = user_row[user_row > 0].index.tolist()
    
    # get the anime IDs that the user has not watched (ratings == 0)
    user_not_watched = user_row[user_row == 0].index.tolist()
    
    # get predicted ratings for the unwatched anime 
    user_pred = R_df.loc[u_id, user_not_watched]
    # R_df contains the predicted ratings for all users and all anime id, but not anime names
    # find top x anime ids with the highest predicted ratings that the user has not watched, and sort them by predicted rating
    # then return the anime names and predicted ratings
    top_x = user_pred.sort_values(ascending=False).head(x)
    anime_ids = top_x.index
    # get the anime names from original_df
    anime_names = df[df['a_id'].isin(anime_ids)]['title'].unique()
    # create a DataFrame with anime names and predicted ratings
    recommendations = pd.DataFrame({
        'title': anime_names,
        'predicted_rating': top_x.values
    })
    
    return recommendations

In [10]:
# read all the datasets
df_1 = pd.read_csv("data/100x100.csv")
df_2 = pd.read_csv("data/100x100_2.csv")

# choose a random user
# chosen_user_1 = np.random.choice(df_1['u_id'].unique(), 1)[0]
# chosen_user_2 = np.random.choice(df_2['u_id'].unique(), 1)[0]

# choose a static user
chosen_user_1 = 966
chosen_user_2 = 966

print(f"Chosen user 1: {chosen_user_1}")
print(f"Chosen user 2: {chosen_user_2}")

Chosen user 1: 966
Chosen user 2: 966


In [14]:
# baseline model x = 10000, using dataset 1
baseline_matrix_1 = baseline_model.baseline_model(10000, df_1)
recommend_anime(baseline_matrix_1, chosen_user_1, df_1)

status: optimal
The predicted matrix is  [[7.27441074 7.27106549 7.26963746 ... 7.2676937  7.27033437 7.27133307]
 [7.27336475 7.27001949 7.26859146 ... 7.2666477  7.26928838 7.27028708]
 [7.27513801 7.27179276 7.27036473 ... 7.26842097 7.27106164 7.27206034]
 ...
 [7.27653232 7.27318706 7.27175903 ... 7.26981528 7.27245595 7.27345465]
 [7.27458299 7.27123773 7.2698097  ... 7.26786595 7.27050662 7.27150532]
 [7.27427275 7.27092749 7.26949947 ... 7.26755571 7.27019638 7.27119508]]


,title,predicted_rating
0,Steins;Gate,7.273055
1,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,7.272548
2,Baccano!,7.272320
3,Psycho-Pass,7.272194
4,Higurashi no Naku Koro ni,7.271803


In [13]:
# baseline model x = 10000, using dataset 2
baseline_matrix_2 = baseline_model.baseline_model(10000, df_2)
recommend_anime(baseline_matrix_2, chosen_user_2, df_2)

status: optimal
The predicted matrix is  [[7.16328422 7.16509824 7.16486639 ... 7.16340096 7.1646664  7.16270116]
 [7.16325405 7.16506808 7.16483623 ... 7.16337079 7.16463624 7.16267099]
 [7.16357602 7.16539004 7.16515819 ... 7.16369276 7.1649582  7.16299295]
 ...
 [7.16313602 7.16495004 7.16471819 ... 7.16325276 7.16451821 7.16255296]
 [7.16097321 7.16278724 7.16255539 ... 7.16108995 7.1623554  7.16039015]
 [7.16186936 7.16368339 7.16345153 ... 7.1619861  7.16325155 7.1612863 ]]


,title,predicted_rating
0,Monster,7.164785
1,Ouran Koukou Host Club,7.164717
2,Gekkan Shoujo Nozaki-kun,7.164386
3,Perfect Blue,7.164353
4,Kaguya-sama wa Kokurasetai: Tensai-tachi no Re...,7.164319


In [15]:
# ALS at rank 20, 1000 iterations, using dataset 1
X, Y = als.als(df_1, 20, 1000)
als_matrix_1 = X @ Y.T
recommend_anime(als_matrix_1, chosen_user_1, df_1)

,title,predicted_rating
0,Mahou Shoujo Madoka★Magica,9.014083
1,Toradora!,8.914269
2,Serial Experiments Lain,8.644720
3,Bakemonogatari,8.643122
4,Nichijou,8.549699


In [16]:
# ALS at rank 20, 1000 iterations, using dataset 2
X, Y = als.als(df_2, 20, 1000)
als_matrix_2 = X @ Y.T
recommend_anime(als_matrix_2, chosen_user_2, df_2)

,title,predicted_rating
0,Ouran Koukou Host Club,8.435496
1,Hanasaku Iroha,8.215526
2,Ginga Eiyuu Densetsu,7.955152
3,Mob Psycho 100 II,7.948599
4,Nisemonogatari,7.800082


In [17]:
# Spectral Regularization Model at lambda 10000, using dataset 1
spectral_matrix_1 = spectral_regularization_model.spectral_regularization_model(10000, df_1)
recommend_anime(spectral_matrix_1, chosen_user_1, df_1)

status: optimal
The predicted matrix is  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 1.50145026e-06 1.50145026e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


,title,predicted_rating
0,Musaigen no Phantom World,0.0
1,Kiznaiver,0.0
2,Charlotte,0.0
3,Netoge no Yome wa Onnanoko ja Nai to Omotta?,0.0
4,Neon Genesis Evangelion,0.0


In [18]:
# Spectral Regularization Model at lambda 10000, using dataset 2
spectral_matrix_2 = spectral_regularization_model.spectral_regularization_model(10000, df_2)
recommend_anime(spectral_matrix_2, chosen_user_2, df_2)

status: optimal
The predicted matrix is  [[0.00000000e+00 1.33468972e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.33468972e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  8.34181090e-07 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.33468972e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.16785352e-06
  1.33468972e-06 0.00000000e+00]]


,title,predicted_rating
0,Monster,0.0
1,Trigun,0.0
2,Mob Psycho 100,0.0
3,Dagashi Kashi,0.0
4,Renai Boukun,0.0


In [20]:
# nuclear norm model, using dataset 1
nuclear_matrix_1 = nuclear_norm_model.nuclear_norm_model_df(df_1)
recommend_anime(nuclear_matrix_1, chosen_user_1, df_1)

Optimization succeeded.


,title,predicted_rating
0,Baccano!,8.684145
1,FLCL,8.511174
2,Lucky☆Star,8.486818
3,Bakemonogatari,8.455556
4,Nichijou,8.293555


In [21]:
# nuclear norm model, using dataset 2
nuclear_matrix_2 = nuclear_norm_model.nuclear_norm_model_df(df_2)
recommend_anime(nuclear_matrix_2, chosen_user_2, df_2)

Optimization succeeded.


,title,predicted_rating
0,Ouran Koukou Host Club,8.459675
1,Nagi no Asu kara,8.223338
2,Hanasaku Iroha,7.961509
3,Mob Psycho 100 II,7.955810
4,Nisemonogatari,7.785970
